In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input-1x28x28| Output-32x28x28| RF-3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input-32x28x28| Output-64x28x28| RF-5x5
        self.pool1 = nn.MaxPool2d(2, 2) #input-64x28x28| Output-64x14x14| RF-10x10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input-64x14x14| Output-128x14x14| RF-12x12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input-128x14x14| Output-256x14x14| RF-14x14
        self.pool2 = nn.MaxPool2d(2, 2) #input-256x14x14| Output-256x7x7| RF-28x28
        self.conv5 = nn.Conv2d(256, 512, 3) #input-256x7x7| Output-512x5x5| RF-30x30
        self.conv6 = nn.Conv2d(512, 1024, 3) #input-512x5x5| Output-1024x3x3| RF-32x32
        self.conv7 = nn.Conv2d(1024, 10, 3) #input-1024x3x3| Output-10x1x1| RF-34x34

    def forward(self, x):  
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # x=conv1(x)->relu->conv2->relu->pool1  [Input =1x28x28 |Output =64x14x14]
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # x=conv3(x)->relu->conv4->relu->pool2  [Input =64x14x14 |Output =256x7x7]
        x = F.relu(self.conv6(F.relu(self.conv5(x)))) # x=conv5(x)->relu->conv6->relu [Input =256x7x7 |Output =1024x3x3]
        x = F.relu(self.conv7(x)) # x=conv7(x)->relu [Input =1024x3x3 |Output =10x1x1]
        x = x.view(-1, 10) #[Input =10x1x1 |Output =1x10]
        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() #check if cuda accelerator is available
device = torch.device("cuda" if use_cuda else "cpu") 
model = Net().to(device)
summary(model, input_size=(1, 28, 28)) #print summary of the model architecture for FloatTensor of shape (C, H, W)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:

# seed the model to obtain consistent results
torch.manual_seed(1)
# This is the batch size(where the whole training dataset is grouped into small batches to train the model).
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# load the training data and perform standard normalization 
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(), #transform an image into a PyTorch tensor, here pixel values are scaled between 0.0 and 1.0.
                        transforms.Normalize((0.1307,), (0.3081,))# parameter for normalization is mean and std dev.
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

#load the test data & perform standard normalization of the data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) #mean = 0.1307 & stdDev = 0.3081 used for normalization
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
from tqdm import tqdm
# Function to train 
'''
Args: 
model : Created model to be used for training
device : GPU or CPU hardware to be used for training
train_loader: data on which the training has to be done
optimizer : Optimization algorithm to be used for minimizing the training loss
epoch : number of epochs(or number of time the model supposed to see the input data)

'''
def train(model, device, train_loader, optimizer, epoch):
    model.train() # Set the model on training mode
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # move the model and data to GPU if CUDA is available, otherwise, it will put them in CPU. 
        optimizer.zero_grad() # Sets gradients of all model parameters to zero
        output = model(data) # Getting the model output
        loss = F.nll_loss(output, target) # calculating the The negative log likelihood loss.It is useful to train a classification problem with C classes.
        loss.backward() # flowing the gradients backward.
        optimizer.step() # parameter updated based on the current gradient.
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


In [9]:
# Function to test 
'''
Args: 
model : created model to be used for training
device : GPU or CPU hardware to be used for training
test_loaded: data on which the testing has to be done
 
'''
def test(model, device, test_loader):
    model.eval() # seting up the model for evalaution.
    test_loss = 0 # setting the test loss to 0
    correct = 0  # counting the no of correct classfication.
    with torch.no_grad(): # turn off gradients, since we are in test mode
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) # copy the data to device.
            output = model(data) # predict the output
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset) # calculating the test loss.

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
model = Net().to(device) # move the model to device.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # initiating the SGD optimizer

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.99245023727417 batch_id=468: 100%|██████████| 469/469 [00:37<00:00, 12.47it/s]



Test set: Average loss: 1.9687, Accuracy: 2790/10000 (28%)

